In [15]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("housing.csv")

In [4]:
# print(df.info())
# df.describe()
# df.hist(bins=50,figsize=(20,15))

In [17]:
df["income_cat"] = pd.cut(df["median_income"],bins=[0.,1.5,3,4.5,6.,np.inf],labels=[1,2,3,4,5])

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)

strat_train , strat_test = None, None
for train_i , test_i in split.split(df,df["income_cat"]):
    strat_train = df.loc[train_i]
    strat_test = df.loc[test_i]

strat_train.drop(columns=["income_cat"],inplace=True)
strat_test.drop(columns=["income_cat"],inplace=True)
strat_train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [6]:
# cp_train = strat_train.copy()
# cp_train.plot(kind="scatter",
#               x="longitude",y="latitude",
#               alpha=0.5,s=cp_train["population"]/100,
#               figsize=(12,10),c="median_house_value",
#              )

In [7]:
cp_train["rooms_per_household"] = cp_train["total_rooms"] / cp_train["households"]
cp_train["bedrooms_per_rooms"] = cp_train["total_bedrooms"] / cp_train["total_rooms"]
cp_train["population_per_household"] = cp_train["population"] / cp_train["households"]
cp_train.corr(numeric_only=True)["median_house_value"].sort_values(ascending=False)

NameError: name 'cp_train' is not defined

In [20]:
X,y = df.drop(columns=["median_house_value"]),df[["median_house_value"]]
X_train,X_test, y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)
# X,y

In [19]:
# handle missing data
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_num = X.select_dtypes(exclude="object")
imputer.fit(X_num)
X_num = imputer.transform(X_num)
X_num = pd.DataFrame(X_num,columns=X.select_dtypes(exclude="object").columns)
X_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20640 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   income_cat          20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [118]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X_cat = X.select_dtypes(include="object")
X_cat = ohe.fit_transform(X_cat)
X_cat.toarray()
X_cat = pd.DataFrame(X_cat.toarray(),columns=ohe.categories_)
X_cat

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...
20635,0.0,1.0,0.0,0.0,0.0
20636,0.0,1.0,0.0,0.0,0.0
20637,0.0,1.0,0.0,0.0,0.0
20638,0.0,1.0,0.0,0.0,0.0


In [21]:
from sklearn.base import BaseEstimator, TransformerMixin


rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombineAttrsTransformer(BaseEstimator,TransformerMixin):
    def __init__(self,add_bed_rooms = True):
        self.add_bed_rooms = add_bed_rooms
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bed_rooms:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        return np.c_[X, rooms_per_household, population_per_household]
# X = X_num.join(X_cat)
# ct = CombineAttrsTransformer()
# X_num = ct.fit_transform(X_num.values)
# X_num


In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_attr = list(X_num.columns)
cat_attr = ['ocean_proximity']


num_pipline = Pipeline([
    ("imputer",SimpleImputer(strategy="median")),
    ("combine_attrs",CombineAttrsTransformer()),
    ("std_scaler",StandardScaler()),
    
])

pipeline = ColumnTransformer([
    ("num",num_pipline,num_attr),
    ("cat",OneHotEncoder(),cat_attr),
])

X_tr =  pipeline.fit_transform(X_train)
X_tr 

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [24]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_tr,y_train.values)

LinearRegression()

In [25]:
some_data = X_train.iloc[:5]
some_labels = y_train.iloc[:5]
some_data_tr = pipeline.transform(some_data)
print(lin_reg.predict(some_data_tr).flatten())
print(some_labels.values.flatten())

[185873.6891356  292091.71095342 238246.24693763 146331.45344945
 166758.93318378]
[103000. 382100. 172600.  93400.  96500.]


In [201]:
from sklearn.metrics import mean_squared_error
y_pred = lin_reg.predict(X_tr)
lin_rmse = np.sqrt(mean_squared_error(y_train,y_pred))
lin_rmse 

67229.89334460018

In [203]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_tr,y_train) 

DecisionTreeRegressor()

In [205]:
y_pred = tree_reg.predict(X_tr)
tree_rmse = np.sqrt(mean_squared_error(y_train,y_pred))
tree_rmse

0.0

In [213]:
from sklearn.model_selection import cross_validate
scores = cross_validate(tree_reg,X_tr,y_train,scoring="neg_mean_squared_error",cv=10)
tree_rsme = np.sqrt(-scores["test_score"])
print(tree_rsme.std())

2098.737131631862


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rand_reg = RandomForestClassifier()
rand_reg.fit(X_tr,y_train.values) 

/home/teagy/.local/lib/python3.10/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
scores = cross_validate(tree_reg,X_tr,y_train,scoring="neg_mean_squared_error",cv=10)
rand_rsme = np.sqrt(-scores["test_score"])
print(scores)
print(rand_rsme.std())